In [ ]:
import polars as pl
import squarify
import rtsvg
rt = rtsvg.RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df            = pl.DataFrame(_lu_)
df            = df.with_columns(pl.col('fm').cast(pl.String), pl.col('to').cast(pl.String))
g             = rt.createNetworkXGraph(df, [('fm','to')])
_nodes_       = list(set(df['fm']) | set(df['to']))
_node_colors_ = {}
for i in range(98): _node_colors_[_nodes_.pop(0)] = 'red'
for i in range(24): _node_colors_[_nodes_.pop(0)] = 'yellow'
for i in range(41): _node_colors_[_nodes_.pop(0)] = 'green'
for i in range(2):  _node_colors_[_nodes_.pop(0)] = 'black'

# Starts Here

_color_to_nodes_ = {}
for _node_ in set(df['fm']) | set(df['to']):
    if _node_ in _node_colors_: _color_ = _node_colors_[_node_]
    else:                       _color_ = rt.co_mgr.getTVColor('data','default')
    if _color_ not in _color_to_nodes_: _color_to_nodes_[_color_] = set()
    _color_to_nodes_[_color_].add(_node_)

_color_size_tuples_ = []
for _color_ in _color_to_nodes_: _color_size_tuples_.append((len(_color_to_nodes_[_color_]), _color_))
_color_size_tuples_.sort(reverse=True)

_color_sizes_ = []
for _tuple_ in _color_size_tuples_: _color_sizes_.append(_tuple_[0])

_normalized_sizes_   = squarify.normalize_sizes(_color_sizes_, 1024, 1024)
_treemap_rectangles_ = squarify.squarify(_normalized_sizes_,0,0,1024,1024)

_pos_ = {}
for i in range(len(_color_size_tuples_)):
    _color_  = _color_size_tuples_[i][1]
    _rect_   = _treemap_rectangles_[i]
    _bounds_ = (_rect_['x'], _rect_['y'], _rect_['x'] + _rect_['dx'], _rect_['y'] + _rect_['dy'])
    rt.rectangularArrangement(g, _color_to_nodes_[_color_], pos=_pos_, bounds=_bounds_)

# Ends Here

rt.link(df, [('fm','to')], pos=_pos_, node_color=_node_colors_, w=512, h=512)
